## Installing Required Dependencies through pip


In [ ]:
!pip install discord
!pip install transformers
!pip install openai
!pip install sentence-transformers

### Note that you may need to restart the system after installing dependencies

---

## Importing Required Dependencies, Updating TinyLlama and Downloading Important Functions.

In [ ]:
# Your OpenAI API Key here
OPENAI_KEY = ""

In [ ]:
# Your Discord Bot token here
DISCORD_BOT_TOKEN = ""

In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_similarity
import json
import gdown
import openai
import json
import numpy as np

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp
!cd llama.cpp && git pull && make clean && LLAMA_CUBLAS=1 make
!pip install -r llama.cpp/requirements.txt

Cloning into 'llama.cpp'...
remote: Enumerating objects: 14540, done.
remote: Counting objects: 100% (14540/14540), done.
remote: Compressing objects: 100% (4366/4366), done.
remote: Total 14540 (delta 10188), reused 14335 (delta 10072), pack-reused 0
Receiving objects: 100% (14540/14540), 16.56 MiB | 15.85 MiB/s, done.
Resolving deltas: 100% (10188/10188), done.
Already up to date.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -Wdouble-promotion 
I CXXFLAGS:  -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread

In [ ]:
MODEL_ID = "mayapati/hidelma"

!git lfs install
!git clone https://huggingface.co/{MODEL_ID}

Git LFS initialized.
Cloning into 'hidelma'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 13 (delta 1), reused 0 (delta 0), pack-reused 3
Unpacking objects: 100% (13/13), 478.34 KiB | 4.69 MiB/s, done.


In [ ]:
MODEL_ID = "TinyLlama/TinyLlama-1.1B-intermediate-step-955k-token-2T"

!git lfs install
!git clone https://huggingface.co/{MODEL_ID}

Git LFS initialized.
Cloning into 'TinyLlama-1.1B-intermediate-step-955k-token-2T'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 22 (delta 5), reused 0 (delta 0), pack-reused 3
Unpacking objects: 100% (22/22), 480.55 KiB | 3.78 MiB/s, done.
Filtering content: 100% (3/3), 201.26 MiB | 1.05 MiB/s, done.
Encountered 2 file(s) that may not have been copied correctly on Windows:
	pytorch_model.bin
	model.safetensors

See: `git lfs help smudge` for more details.


In [ ]:
MODEL_ID = "mayapati/hidelma"
MODEL_NAME = MODEL_ID.split('/')[-1]
fp16 = f"{MODEL_NAME}/{MODEL_NAME.lower()}.fp16.bin"
!python llama.cpp/convert.py {MODEL_NAME} --outtype f16 --outfile {fp16} --vocab-dir "/content/TinyLlama-1.1B-intermediate-step-955k-token-2T"


Loading model file hidelma/pytorch_model.bin
params = Params(n_vocab=32000, n_embd=2048, n_layer=22, n_ctx=2048, n_ff=5632, n_head=32, n_head_kv=4, n_experts=None, n_experts_used=None, f_norm_eps=1e-05, rope_scaling_type=None, f_rope_freq_base=10000.0, f_rope_scale=None, n_orig_ctx=None, rope_finetuned=None, ftype=<GGMLFileType.MostlyF16: 1>, path_model=PosixPath('hidelma'))
32000 32000
Vocab info: <VocabLoader with 32000 base tokens and 0 added tokens>
Special vocab info: <SpecialVocab with 61249 merges, special tokens {'bos': 1, 'eos': 2, 'unk': 0, 'pad': 2}, add special tokens unset>
Permuting layer 0
Permuting layer 1
Permuting layer 2
Permuting layer 3
Permuting layer 4
Permuting layer 5
Permuting layer 6
Permuting layer 7
Permuting layer 8
Permuting layer 9
Permuting layer 10
Permuting layer 11
Permuting layer 12
Permuting layer 13
Permuting layer 14
Permuting layer 15
Permuting layer 16
Permuting layer 17
Permuting layer 18
Permuting layer 19
Permuting layer 20
Permuting layer 2

In [ ]:
QUANTIZATION_METHODS = ["q5_k_m"]

for method in QUANTIZATION_METHODS:
    qtype = f"{MODEL_NAME}/{MODEL_NAME.lower()}.{method.upper()}.gguf"
    !./llama.cpp/quantize {fp16} {qtype} {method}

main: build = 1681 (c7e9701)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing 'hidelma/hidelma.fp16.bin' to 'hidelma/hidelma.Q5_K_M.gguf' as Q5_K_M
llama_model_loader: loaded meta data with 21 key-value pairs and 201 tensors from hidelma/hidelma.fp16.bin (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                       llama.context_length u32              = 2048
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 2048
llama_model_loader: - kv   4:                          llama.block_count u32              = 22
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 

In [ ]:
!pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 57.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.24-cp310-cp310-manylinux_2_35_x86_64.whl size=2087880 sha256=d25ac24aae55b96f0aa88e2d074c5f311a1a43227e52fe736b6b2b8e1bdf918d
  Stored in directory: /root/.cache/pip/wheels/9b/5f/31/172f820a61d82eb7b057b7d05ff9a05345012066e8ae781788
Successfully built llama-cpp-python


In [ ]:
Retriver_model_url='https://drive.google.com/drive/folders/11DeDLA5nWss5CWU5-bvHmw77bhXlFrNO'
gdown.download_folder(Retriver_model_url, quiet=True, use_cookies=False)
gdown.download("https://drive.google.com/uc?id=1AKHYcS-bFnpeN_f2X57LBW7EhaRuVXiA", quiet=True, use_cookies=False)

'final_tool_database.json'

## Defining builder functions

In [ ]:
from transformers import StoppingCriteria
import torch

class EosListStoppingCriteria(StoppingCriteria):
    def _init_(self, eos_sequence = [518, 29914, 25580, 29962]):
        self.eos_sequence = eos_sequence

    def _call_(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        last_ids = input_ids[:,-len(self.eos_sequence):].tolist()
        return self.eos_sequence in last_ids

In [ ]:
def get_embeddings(texts):
    response = openai.Embedding.create(
        model="text-embedding-ada-002",
        input=texts
    )
    return [embedding['embedding'] for embedding in response['data']]

In [ ]:
from llama_cpp import Llama

# Make sure to change the path according to work environment. The paths should "Current_Directory/checkpoint_epoch_4.pt" and "Current_Directory/hidelma/hidelma.Q5_K_M.gguf"
model = SentenceTransformer('/content/checkpoint_epoch_4.pt')
LLM = Llama(model_path="/content/hidelma/hidelma.Q5_K_M.gguf", stopping_criteria = [EosListStoppingCriteria()])

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


## Start of AI-Agent-007 ToolBot Pipeline

### Example to improve GPT Prompting

In [ ]:
example="""
example query 1: Prioritize my P0 issues and add them to the current sprint

THOUGHT_ACTION_PAIR_1:
thought: The user query is asking to prioritize user's P0 issues and add them to the current sprint, I will first call the tool "whoami" to get the user ID.
action: Using "whoami" to return the ID of the current user.
JSON:
{
    "tool_name": "whoami",
    "arguments": []
}

THOUGHT_ACTION_PAIR_2:
thought: Next I will call the "work_list" tool to find the P0 issue of the current user.
action: Using the "work_list" tool.
JSON:
{
    "tool_name": "works_list",
    "arguments":
    [
        {
            "argument_name": "issue.priority",
            "argument_value": ["p0"]
        },
        {
            "argument_name": "owned_by",
            "argument_value": ["$$PREV[0]"]
        },
        {
            "argument_name": "type",
            "argument_value": ["issue"]
        }
    ]
}

THOUGHT_ACTION_PAIR_3:
thought: Then to prioritize the P0 issues, I will call the "prioritize_objects" tool.
action: Check if the 'prioritize_objects' tool is available.
JSON:
{
    "tool_name": "prioritize_objects",
    "arguments":
    [
        {
            "argument_name": "objects",
            "argument_value": "$$PREV[1]"
        }
    ]
}

THOUGHT_ACTION_PAIR_4:
thought: Now to get the current sprint ID, I will use "get_sprint_id".
action: The 'get_sprint_id' tool can be used to retrieve the current sprint ID.
JSON:
{
    "tool_name": "get_sprint_id",
    "arguments": []
}

THOUGHT_ACTION_PAIR_5:
thought: Then to add the work items to the current sprint, i call 'add_work_items_to_sprint'.
action: Check if the 'add_work_items_to_sprint' tool is available.
JSON:
{
  "tool_name": "add_work_items_to_sprint",
  "arguments": [
    {
      "argument_name": "work_ids",
      "argument_value": "$$PREV[2]"
    },
    {
      "argument_name": "sprint_id",
      "argument_value": "$$PREV[3]"
    }
  ]
}

Expected_output:
[
    {
        "tool_name": "whoami",
        "arguments": []
    },
    {
        "tool_name": "works_list",
        "arguments":
        [
            {
                "argument_name": "issue.priority",
                "argument_value": ["p0"]
            },
            {
                "argument_name": "owned_by",
                "argument_value": ["$$PREV[0]"]
            },
            {
                "argument_name": "type",
                "argument_value": ["issue"]
            }
        ]
    },
    {
        "tool_name": "prioritize_objects",
        "arguments":
        [
            {
                "argument_name": "objects",
                "argument_value": "$$PREV[1]"
            }
        ]
    },
    {
        "tool_name": "get_sprint_id",
        "arguments": []
    },
    {
        "tool_name": "add_work_items_to_sprint",
        "arguments":
        [
            {
                "argument_name": "work_ids",
                "argument_value": "$$PREV[2]"
            },
            {
                "argument_name": "sprint_id",
                "argument_value": "$$PREV[3]"
            }
        ]
    }
]
END_JSON
**************************************************"""

## Regex Parser for extracting precise JSON from Chain of Thoughts

In [ ]:
# def extract_json_from_string(input_string):
#     import json
#     import re

#     match = re.search(r'output:', input_string, re.IGNORECASE)
#     if match:
#         start_pos = match.end()
#         try:
#             json_data = input_string[start_pos:].strip()
#             parsed_json = json.loads(json_data)
#             return json.dumps(parsed_json, indent=4)
#         except json.JSONDecodeError:
#             return "Invalid JSON format in the input string."
#     else:
#         return "No JSON data found in the input string."



def extract_json_with_fallback(input_string):
    match_start = re.search(r'output:', input_string, re.IGNORECASE)
    match_end = re.search(r'END_JSON', input_string, re.IGNORECASE)

    if match_start:
        start_pos = match_start.end()
        if match_end:
            end_pos = match_end.start()
            json_data = input_string[start_pos:end_pos].strip()
        else:
            json_data = input_string[start_pos:].strip()

        try:
            parsed_json = json.loads(json_data)
            return json.dumps(parsed_json, indent=4)
        except json.JSONDecodeError:
            return "Invalid JSON format in the input string."
    else:
        return "No 'output:' marker found in the input string."

## Extractor Function for removing TinyLlama cache and generating precise Chain of Thought (COT)

In [ ]:
def extract_string(input_string):
    inst1_index = input_string.find("[/INST]")
    inst2_index = input_string.find(" <</INST>>")

    if inst1_index == -1 and inst2_index == -1:
        return input_string

    if inst1_index == -1 or (inst2_index != -1 and inst2_index < inst1_index):
        return input_string[:inst2_index]
    else:
        return input_string[:inst1_index]


## ToolBot hosted on our very own Discord Server

In [ ]:
import discord
from discord.ext import commands
import asyncio
import nest_asyncio
import openai
import logging
import os
import time
import json

logging.basicConfig(level=logging.INFO)
nest_asyncio.apply()

intents = discord.Intents.default()
intents.messages = True
intents.message_content = True

is_processing_request = False

bot = commands.Bot(command_prefix="!", intents=intents)

openai.api_key = OPENAI_KEY

conversation_history = {}

def recursive_call(response,query,relevant_api):
  retry = f"""
  This is the {response} for the following Query: {query} and these are the tools : {relevant_api}.
   Generate an alternative response which solves the query. Follow the same structure as the initial response: {response}.
  """

  recursive_response = openai.ChatCompletion.create(
            model="gpt-4-1106-preview",
            messages=[{"role": "system", "content": "You are a helpful assistant."},
                      {"role": "user", "content": retry}],
            temperature = 0.2
        )
  time.sleep(6)
  query_recursive = f'''
    For this User Query: {query}
    we now have the following response:
    {recursive_response.choices[0].message['content']}

    Now analyse this response and check whether the generated APIs are actually present in relevant_api:{relevant_api}.
    If you find that some APIs are not present in {relevant_api} or you think the response generated does not solve the problem return "NO" else return "YES". Make sure to answer in one word.
    '''
  time.sleep(6)
  response2 = openai.ChatCompletion.create(
            model="gpt-4-1106-preview",
            messages=[{"role": "system", "content": "You are a helpful assistant."},
                      {"role": "user", "content": query_recursive}],
            temperature = 0.2
        )


  if (response2.choices[0].message["content"].lower())[0] == "y":
            return recursive_response.choices[0].message['content']
  else:
          return "[]"

@bot.event
async def on_ready():
    print(f"Logged in as {bot.user.name}")

async def write_response_to_file(response, file_path):
    with open(file_path, 'w') as file:
        file.write(response)

async def generate_response(prompt):
    try:
            query = prompt


            system_message_cot = "Given a user query in English, you will respond with a well-reasoned response"
            prompt_cot = f"[INST] <<SYS>>\n{system_message_cot}\n<</SYS>>\n\n {query} [/INST]"
            # output_cot = LLM(prompt)

            # input_str = "This is a sample string [/INST] with some text. <</INST>> More text here."
            # result_cot = extract_string(output_cot["choices"][0]['text'])

            query_api_description = []

            """
            This code is for combining a self-trained IR Bert Retriever. Use this if you want more generalised approach
            towards tool-retrieval. However it has been noticed that ada-002(OpenAI Embeddings) performs better in terms
            of focused set of tools for better accuracy.
            """
            '''

            final_ = []
            rel_tools = []
            with open('/content/final_tool_database.json', 'r') as file:
                json_list = json.load(file)
            for tool in json_list:
              query_api_description += [[query,tool["api_description"]]]


            for i in range(len(query_api_description)):
              encodings = [model.encode(query_api_description[i])]
              similarity = cosine_similarity(encodings[0][0].reshape(1,-1), encodings[0][1].reshape(1,-1))
              final_ += [((similarity[0][0]),(json_list[i]["tool_name"]))]

            sorted_data = sorted(final_, key=lambda x: x[0],reverse=True)

            for i in range(0,min(len(sorted_data),10)):
              rel_tools+=[sorted_data[i][1]]

            final_list = []

            for tool in json_list:
                tool_name = tool["tool_name"]
                if tool_name in rel_tools:
                  final_list+=[tool]

            relevant_api = f"""{final_list}
            """

            '''

            final_ = []
            rel_tools = []
            with open('/content/final_tool_database.json', 'r') as file:
                json_list = json.load(file)

            query_api_description = []
            for tool in json_list:
                query_api_description.append([query, tool["api_description"]])

            all_texts = [item for sublist in query_api_description for item in sublist]
            embeddings = get_embeddings(all_texts)

            for i in range(0, len(query_api_description), 2):
                similarity = cosine_similarity(
                    [embeddings[i]], [embeddings[i + 1]]
                )
                final_.append((similarity[0][0], json_list[i//2]["tool_name"]))

            sorted_data = sorted(final_, key=lambda x: x[0], reverse=True)
            rel_tools = [tool[1] for tool in sorted_data[:min(10, len(sorted_data))]]

            final_list = [tool for tool in json_list if tool["tool_name"] in rel_tools]

            relevant_api = f"{final_list}"
            print(relevant_api)
            system_prompt = f"""Given a user query and an available list of relevant tools with the tool arguments.
        a- Dissect the user query into smaller, manageable subproblems that lead towards a solution.
        b- For each subproblem, create a sequence of thoughts, actions, and JSON API calls. Analyze each step: If a subproblem cannot be solved with the available tools, return "not possible."
          remember
        c- the Final_output should be the list of all the JSON in the previous steps if you can solve the problem otherwise it should be an empty list
          format: Final_output=[{{....}},{{....}}.....] or Final_output=[] depending on whether you can solve the question or not
          the Final_output list should be non-empty if you fully solve the problem
        1. For each thought-action pair, analyze if the tool execution will be possible or not. Assume the output for each thought-action using the given list of tools.
        2. Match the argument_value given by you with the specified argument type for the tool. If the argument values and argument types match then assume that the function call is possible and will be successful, otherwise end the thought-action sequence and return not possible.
          remember that you shouldn't change any data type of the query all the items in user queries are strings
          and the output of the API you can assume using the description whether it is an object or a string or any other
        3. If the argument_values and argument_type matches then check the logic for the argument, the argument_value should be according to the argument_description. If the argument values and argument types match then assume that the function call is possible and will be successful, otherwise end the thought-action sequence and return not possible.

        For each thought-action pair generate the following:
        JSON

        “tool_name”: name of the TOOL from the TOOL list.
        “arguments”: [
        {{
            “argument_name”: Name of TOOL’s argument,
            “argument_value”: Value that the argument takes according to the user query. It should have the correct Argument type. Fill according to the argument description and use the Argument Value Example for sample help
        }}]
        4. After the generation of thought-action JSON, try to analyze whether the subproblem
        5. The output is a list of argument names called in a combination. Output from one argument can be given as input to other arguments in the tool. To reference the value of the ith argument in the chain, use $$PREV[i] as argument value. i = 0, 1, .. j-1; j = current arguments’s index in the array. This should be correct according to the thought-action sequence.
        6. One tool can be used multiple times in the same output with different argument values
        7. Do not hallucinate new argument names. The argument value should be in specified format for that argument only
        8. If you are going to use a previous value as an argument then check whether the type of the argument is accepted by the argument_value you are using it in
        9. If some information regarding me is required then you can use the tool "who_am_i" if it is available in the list of tools. If it is not present DO NOT USE IT.
        10. Give me the sequence of thought-action JSON pairs in the following format for the ith thought-action pair
        THOUGHT_ACTION_PAIR_i: {{
            thought:  ,
            2-action: ,
            3-JSON: ,
            4-analysis:
        }}

        11. If some information about a 'customer' is asked then you need to use tools which can provide information about a customer for ex "search_object_by_name". You can use output of that tool for other tools later.
        12. If the current sprint is required then use the tool "get_sprint_id" if it is available in the given set of tools. If it is not present then DO NOT use it. If a similar tool is present you can use that
        13.If the question is a physchological, philosophical or general knowledge question not related to an organisation then return an empty list: [] and give Expected_output as "ORG_NOT_POSSIBLE"
        Final output given by you should be a list of thought and action pairs in natural language, response for the thought-action pair in the JSON schema specified, if response is not possible by matching argument types with the type of argument value returned by you or the argument_value is not matching the argument_description then return NOT_POSSIBLE.
        Return the final json after "Expected_output" and add "END_JSON" after final output json is complete. THIS IS VERY IMPORTANT
        REMEMBER:- you have to use the tools name given below only you cannot generate your new tools.
        Relevant Tools List:
        {relevant_api}
          [if no tool present in the list then you ARE not required to use them]
        User Query: {query}
        i am giving you a set of example you can refer them to get the basic stucture of  what kind of thinking process to adopt.YOU CAN ONLY TAKE INSPIRATION DO NOT COPY THEM THESE ARE ONLY FOR YOUR REFERENCE!!
        examples:{example}
        Generate output using only the given tools for the User Query. DO NOT HALLUCINATE NEW TOOLS, ONLY USE THE TOOLS PROVIDED

        """
# You also can refer to a Guide:{result_cot}
            response = openai.ChatCompletion.create(
                model="gpt-4-1106-preview",
                messages=[{"role": "system", "content": "You are a helpful assistant."},
                          {"role": "user", "content": system_prompt}],
                temperature = 0.2
            )

            response_content = response.choices[0].message['content']

            response_content_no_spaces = response_content.replace(" ", "")
            contains_org_not_possible = "ORG_NOT_POSSIBLE" in response_content_no_spaces
            if(contains_org_not_possible):
              print("YAYA")
              await write_response_to_file("[]", '/content/response.txt')
              return '/content/response.txt'

            print("#########"+response_content)

            time.sleep(6)

            query2 = f'''
        For this User Query: {query}
        we now have the following response to the query:
        {response.choices[0].message['content']}

        Now analyse this response and check whether the generated APIs are actually present in relevant_api:{relevant_api}.
        If you find that APIs are not present in {relevant_api} or you think the answer does not solve the problem return "NO" else return "YES". Make sure to answer in one word.
        '''
            response2 = openai.ChatCompletion.create(
                model="gpt-4-1106-preview",
                messages=[{"role": "system", "content": "You are a helpful assistant."},
                          {"role": "user", "content": query2}],
                temperature = 0.2
            )
            time.sleep(6)
            print("############"+response2.choices[0].message["content"].lower())
            if (response2.choices[0].message["content"].lower())[0] == "y":
              if(response.choices[0].message['content']=="[]"):
                await write_response_to_file("[]", '/content/response.txt')
              else:
                output = extract_json_from_string(response.choices[0].message['content'])
                await write_response_to_file(output, '/content/response.txt')

              return '/content/response.txt'
            else:
              x = recursive_call(response.choices[0].message['content'],query,relevant_api)
              if(x=="[]"):
                await write_response_to_file(x, '/content/response.txt')
              else:
                output = extract_json_from_string(x)
                await write_response_to_file(x, '/content/response.txt')

              return '/content/response.txt'


    except Exception as e:
        print(f"Error: {e}")
        return "An error occurred while generating the response."

@bot.command()
async def talk(ctx, *, prompt):
    global is_processing_request
    global conversation_history

    if is_processing_request:
        return
    is_processing_request = True

    user_id = ctx.message.author.id
    if user_id not in conversation_history:
        conversation_history[user_id] = []

    try:
        check = prompt.lower()[0:3]
        if(check.startswith("q:")):

            full_conversation = conversation_history[user_id][-2:]
            processing_message = await ctx.send("Processing your request...")
            response_file_path = await generate_response(prompt[2:]+ "Previous Response:"+str(full_conversation))

            with open(response_file_path, 'rb') as file:
                await ctx.send(file=discord.File(file, 'response.txt'))
                bot_response = file.read().decode('utf-8')

            conversation_history[user_id].append({"role": "user", "content": prompt[2:]})
            conversation_history[user_id].append({"role": "system", "content": bot_response})

            print(conversation_history)
            await processing_message.delete()
        else:
            full_conversation = conversation_history[user_id][-2:]
            processing_message = await ctx.send("Processing your request...")

            response = openai.ChatCompletion.create(
                model="gpt-4-1106-preview",
                messages=[{"role": "system", "content": "You are a helpful assistant."+ "Previous Response:"+str(full_conversation)}] ,
                temperature=0.2
            )

            bot_response = response.choices[0].message['content']
            conversation_history[user_id].append({"role": "system", "content": bot_response})
            print(conversation_history)

            await processing_message.edit(content=bot_response)

    except Exception as e:
        await ctx.send(f"An error occurred: {e}")
    finally:
        is_processing_request = False


if __name__ == "__main__":
  loop = asyncio.get_event_loop()
  try:
      bot.run(DISCORD_BOT_TOKEN)
  except KeyboardInterrupt:
      loop.run_until_complete(bot.logout())
      loop.close()


2023-12-22 07:15:11 INFO     discord.client logging in using static token
2023-12-22 07:15:11 INFO     discord.client logging in using static token
2023-12-22 07:15:11 INFO     discord.client logging in using static token
2023-12-22 07:15:11 INFO     discord.client logging in using static token
2023-12-22 07:15:11 INFO     discord.client logging in using static token
INFO:discord.client:logging in using static token
2023-12-22 07:15:11 INFO     discord.gateway Shard ID None has connected to Gateway (Session ID: d4b64d6e209df8112a5c440a318349a1).
2023-12-22 07:15:11 INFO     discord.gateway Shard ID None has connected to Gateway (Session ID: d4b64d6e209df8112a5c440a318349a1).
2023-12-22 07:15:11 INFO     discord.gateway Shard ID None has connected to Gateway (Session ID: d4b64d6e209df8112a5c440a318349a1).
2023-12-22 07:15:11 INFO     discord.gateway Shard ID None has connected to Gateway (Session ID: d4b64d6e209df8112a5c440a318349a1).
2023-12-22 07:15:11 INFO     discord.gateway Shard I

Logged in as ToolBot59DevRev
[{'tool_name': 'works_list', 'api_description': 'A flexible tool to fetch a customized list of work items, allowing users to apply various filters such as parts, user creation, priorities, organizational association, ownership, stages, response requirements, and severity levels. It supports an array of argument types for intricate filtering and a limit feature for controlling the output size. Ideal for streamlined task and ticket management in diverse organizational contexts.', 'required_parameters': [{'name': 'applies_to_part', 'type': 'array of strings', 'description': 'Selects work items associated with any of the specified parts.'}, {'name': 'created_by', 'type': 'array of strings', 'description': 'Targets work items created by the listed users.'}, {'name': 'issue.priority', 'type': 'array of strings', 'description': 'Focuses on issues of specified priorities, such as p0, p1, p2, p3.'}, {'name': 'issue.rev_orgs', 'type': 'array of strings', 'description

2023-12-22 07:16:02 WARNING  discord.gateway Shard ID None heartbeat blocked for more than 10 seconds.
Loop thread traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever
    self._run_once()
  File "/usr/local/lib/python3.10

############yes
{1072768125327659040: [{'role': 'user', 'content': ' Given a customer meeting transcript T, create action items and add them to my current sprint'}, {'role': 'system', 'content': ''}]}


2023-12-22 07:16:12 INFO     discord.gateway Shard ID None has successfully RESUMED session d4b64d6e209df8112a5c440a318349a1.
2023-12-22 07:16:12 INFO     discord.gateway Shard ID None has successfully RESUMED session d4b64d6e209df8112a5c440a318349a1.
2023-12-22 07:16:12 INFO     discord.gateway Shard ID None has successfully RESUMED session d4b64d6e209df8112a5c440a318349a1.
2023-12-22 07:16:12 INFO     discord.gateway Shard ID None has successfully RESUMED session d4b64d6e209df8112a5c440a318349a1.
2023-12-22 07:16:12 INFO     discord.gateway Shard ID None has successfully RESUMED session d4b64d6e209df8112a5c440a318349a1.
INFO:discord.gateway:Shard ID None has successfully RESUMED session d4b64d6e209df8112a5c440a318349a1.
